# Cargando las librerias requeridas

In [ ]:
#Instalando las librerias requeridas
# import sys
# !{sys.executable} -m pip install numpy, pandas, plotnine, sklearn

In [ ]:
import numpy as np
import pandas as pd
import plotnine as ptn
from plotnine import *

In [ ]:
import sys
print("Python version:",sys.version)
print("Numpy version:",np.__version__)
print("Pandas version:",pd.__version__)
print("Plotnine version:",ptn.__version__)


In [ ]:
from sklearn import datasets
iris = datasets.load_iris() #Cargando el dataset de la flor de iris

tempDF=pd.DataFrame(iris.data,columns=iris.feature_names)


In [ ]:
ggplot(tempDF, aes(x="sepal length (cm)", y="sepal width (cm)")) + geom_point()

In [ ]:
ggplot(tempDF, aes(x="petal length (cm)", y="petal width (cm)")) + geom_point()

# Modelos de la clase sklearn
La mayoría de los modelos de sklearn corresponden a “objetos” de cierta “clase” con propiedades y funciones específicas. En términos prácticos solo se necesita crear una caja mágica que contiene todo lo necesario para aplicar y analizar los modelos.
* from sklearn.xxx import modeloDeseado => Importación del modelo
* nuevoModelo = modeloDeseado() => Creación del modelo
* nuevoModelo.function() => Aplicar alguna función
* nuevoModelo.attribute => Ver los valores de un atributo

# Clase k-means

Existen múltiples parámetros para el modelo k-means:<br>
KMeans(n_clusters=8, init=’k-means++’, n_init=10, max_iter=300, tol=0.0001, random_state=None)
* n_clusters => número de clusters
* init => ’k-means++’ inicialización inteligente, 'random' aleatoria 
* n_init => Número de veces que aplicaremos k-means
* max_iter => Máximo número de iteraciones para cada ejecución
* tol => Tolerancia para la convergencia
* random_state => Semilla para inicializar los centroides. Use un entero para ser determinista.

Existen múltiple métodos/funciones para el modelo k-means
* km.fit(data) => entrenar el modelo usando ciertos datos, retorna el modelo entrenado
* km.predict(data) => dado un modelo entrenado, determina a que clase pertenece cada punto, retorna un vector con predicciones

In [ ]:
from sklearn.cluster import KMeans
#K-means es una clase con múltiples parámetros, atributos y funciones.
#Para aplicar k-means es necesario crear el objeto y luego aplicarto

#Creando un objeto de k-means con las condiciones iniciales
km = KMeans(n_clusters=4)
#El objeto ha sido creado

#Entrenando el modelo con los datos de iris
km=km.fit(tempDF)

# Caracteristicas del modelo entrenado
Una vez entrenado el modelo, existen nuevas características que podemos observar (atributos)<br>
Atributos
* cluster_centers: Las coordenadas de los centroides. Si el algoritmo no converge estos no serán consistentes con los labels.
* inertia_float: la suma total de los within cluster distance
* labels: Las etiquetas de cada punto (cluster al que pertenece)
* n_iter_: número de iteraciones del algoritmo

In [ ]:
#El modelo ya ha sido entrenado, veamos los resultados
print(km.cluster_centers_) #Mostrando los centros de los clusters

print(km.inertia_) #Within cluster distance

print(km.labels_) #Etiquetas

print(km.n_iter_) #Número de iteraciones del algoritmo

# Graficando los clusters

In [ ]:
labels = pd.Series(km.labels_, index=tempDF.index, dtype='category')

In [ ]:
ggplot(tempDF, aes(x="sepal length (cm)", y="sepal width (cm)", color=labels)) + geom_point()

In [ ]:
ggplot(tempDF, aes(x="petal length (cm)", y="petal width (cm)", color=labels)) + geom_point()

In [ ]:
km.cluster_centers_

# Buscando el valor de K

In [ ]:
sse = [] #Variable para guardar el within cluster distance
numK = 18
for k in range(1, numK): #número de cluster a probar
    #Creando el modelo, entrenandolo y obteniendo el valor del within cluster distance
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(tempDF)
    sse.append(kmeans.inertia_)
# #graficando
# tempDataFrame=pd.DataFrame(range(1, numK),columns=["numK"])
# tempDataFrame["sse"]=sse
# (ggplot(tempDataFrame)+aes(x="numK",y="sse")+theme_bw()+geom_line()+labs(x="Número de clusters",y="WCD")
#   +scale_x_continuous(breaks=range(1,numK)))

In [ ]:
pd.DataFrame(sse, index=np.arange(1,numK)).plot()

# Buscando el valor de K escalando los datos y multiples iteraciones

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #Creamos un objeto de la clase StandardScaler
scaled_features = scaler.fit_transform(tempDF) #estandarización los datos (media 0, varianza 1)
pd.DataFrame(scaled_features, columns=tempDF.columns)

In [ ]:
#Aplicando el mismo proceso anterior para buscar el número de K
sse = []
numK = 18
for k in range(1, numK):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

pd.DataFrame(sse, index=np.arange(1,numK)).plot()

# K-modes
Parametros
* n_clusters: número de clusters.
* max_iter: máximo número de iteraciones.
* init: Método de inicialización ('Huang', 'Cao', 'random') default: 'Cao'
* n_init: número de veces que K-modes se ejecuta

Al igual que k-means tiene las funciones fit and predict

Atributos
* cluster_centroids_: Centroide de los clusters
* labels_: Etiqueta de cada punto
* cost_: Suma de todos los puntos con respecto a los centroides
* n_iter_: número de iteraciones para el algoritmo

In [ ]:
#Instalando una nueva libreria
# import sys
# !{sys.executable} -m pip install kmodes
from kmodes.kmodes import KModes

In [ ]:
#Cargando y viendo los datos
#En el caso de google colab tiene que subir el archivo a la nube primero
bank = pd.read_csv('./data/bank.csv',sep=";")
#https://archive.ics.uci.edu/ml/datasets/bank+marketing#
#Es un subset de esta base de datos



In [ ]:
bank

In [ ]:

#Extrayendo las variables nominales
bank_cust = bank[['job', 'marital', 'education', 'default', 'housing', 'loan','contact','month','poutcome']].copy()
bank_cust


In [ ]:
#Agregando edad como variable categorica
bank_cust['edad'] = pd.cut(bank['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                                 labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])
bank_cust

In [ ]:
bank_cust.describe() #Viendo el resumen de los datos

In [ ]:
#Transformando los datos a números (sklearn no trabaja con datos categóricos)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(bank_cust['job'])

In [ ]:
bank_cust_orig=bank_cust.copy() #Copia de los datos de origen

le = preprocessing.LabelEncoder()
bank_cust = bank_cust.apply(le.fit_transform)
bank_cust

In [ ]:
#Creando el modelo y ajustándolo
km_cao = KModes(n_clusters=3, init = "Cao", n_init = 1, verbose=1)
km_cao = km_cao.fit(bank_cust)
print("Cluster 0:",sum(km_cao.labels_==0))
print("Cluster 1:",sum(km_cao.labels_==1))
print("Cluster 2:",sum(km_cao.labels_==2))

In [ ]:
#Veamos los centroides 
out = pd.DataFrame()
for col in range(len(bank_cust.columns)):
    le.fit(bank_cust_orig.iloc[:,col])
    out[bank_cust.columns[col]] = le.inverse_transform(km_cao.cluster_centroids_[:,col])

In [ ]:
out

# Interpretando el modelo

In [ ]:
from sklearn import datasets
wine = datasets.load_wine() #Cargando el dataset de vino

tempDataFrame=pd.DataFrame(wine.data, columns=wine.feature_names)
tempDataFrame

In [ ]:
#Escalando los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #Creamos un objeto de la clase StandardScaler
scaled_features = scaler.fit_transform(wine.data) #Transformamos los datos
sse=[]

#Buscando el número de clusters
from sklearn.cluster import KMeans
maxCluster=40
for k in range(1, maxCluster):
    kmeans = KMeans(n_clusters=k,n_init=10)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)
# tempDataFrame=pd.DataFrame(range(1, maxCluster),columns=["numK"])
# tempDataFrame["sse"]=sse
# (ggplot(tempDataFrame)+aes(x="numK",y="sse")+theme_bw()+geom_line()+labs(x="Número de clusters",y="WCD")
#  +scale_x_continuous(breaks=range(1,maxCluster)))
pd.DataFrame(sse, index=np.arange(1,maxCluster)).plot()

In [ ]:
#Aplicando el número de clusters seleccionado
selectedCluster=3
kmeans = KMeans(n_clusters=selectedCluster,n_init=10)
kmeans.fit(scaled_features)

# Interpretando el modelo
## Árbol de clasificación

In [ ]:
#Posteriormente veremos y entenderemos más sobre el árbol de decisión
import matplotlib.pyplot as plt

#Creando un objeto de la clase tree
from sklearn import tree

tempDF=pd.DataFrame(wine.data,columns=wine.feature_names)
clf = tree.DecisionTreeClassifier(max_depth=2)
clf = clf.fit(tempDF, kmeans.labels_.astype("str"))

plt.figure(figsize=(15,15))
tree.plot_tree(clf,feature_names = wine.feature_names,filled=True) 
plt.show()

In [ ]:
tempDF.describe()

# Interpretando el modelo
## PCA

In [ ]:
#Creando el objeto y aplicando PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(scaled_features)
tempData = pca.transform(scaled_features)
tempData = pd.DataFrame(tempData,columns=["PC1","PC2"])
tempData["labels"]=kmeans.labels_
tempData
ggplot(tempData)+aes(x="PC1",y="PC2",color="factor(labels)")+geom_point()+theme_bw()

In [ ]:
#Analizando los componentes
datapc = pd.DataFrame(pca.components_.transpose(),columns=["PC1","PC2"])
datapc['varNames']=wine.feature_names
datapc
(ggplot(datapc)+
  theme_bw()+
  labs(title="loading plots")+
  geom_text(aes(x="PC1", y="PC2", label="varNames"), size = 10, color="blue")+
  geom_segment(aes(x=0, y=0, xend="PC1", yend="PC2"), arrow=arrow(length=0.1),size=1, alpha=0.75, color="blue")+
  scale_x_continuous(limits=[-0.9,0.9],breaks=[-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3,0.4,0.5,0.6])+
  scale_y_continuous(limits=[-0.9,0.9],breaks=[-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3,0.4,0.5,0.6])+
  geom_hline(yintercept = 0, size=.2)+
  geom_vline(xintercept = 0, size=.2)+
  coord_cartesian([-0.6,0.6],[-0.6,0.6])
)

# Interpretando el modelo
## Análisis de centroides

In [ ]:
KMcentroids=pd.DataFrame(kmeans.cluster_centers_,columns=wine.feature_names)
KMcentroids

In [ ]:
KMcentroids

In [ ]:
tempDF=pd.DataFrame({'varNames': KMcentroids.columns, 'std': KMcentroids.std()})
#tempDF=pd.DataFrame(KMcentroids.std(),columns=["std"])
orderVar=tempDF.sort_values("std")["varNames"]



In [ ]:
ggplot(tempDF, aes(x='varNames', y='std')) + geom_col() + coord_flip() \
 +scale_x_discrete(limits=orderVar)  +labs(title="desviación de los centroides",x="")





In [ ]:
dfCentroids = pd.DataFrame(scaler.inverse_transform(KMcentroids), columns=KMcentroids.columns)

In [ ]:
df=pd.DataFrame(wine.data, columns=wine.feature_names)
df['cluster'] = kmeans.labels_


In [ ]:
df2 = df.melt(id_vars='cluster')

In [ ]:
ggplot(df2[df2['variable']=='ash'], aes(x='factor(cluster)', y='value')) \
    + stat_summary(geom='geom_errorbar') +stat_summary(geom='point') + facet_wrap('variable', scales='free_y')

In [ ]:
ggplot(df2, aes(x='factor(cluster)', y='value')) \
    + stat_summary(geom='geom_errorbar') +stat_summary(geom='point') \
    + facet_wrap('variable', scales='free_y')  + theme(figure_size=(10,10),subplots_adjust={'wspace': 0.25}) 